# Read Pdf and convert into text

In [1]:
from langchain.document_loaders import PyPDFLoader

# Path to your PDF file
pdf_path = r"C:\Users\ahmed\Downloads\Documents\IZT_WB117.pdf"

# Initialize the PDF Loader
loader = PyPDFLoader(pdf_path)

# Load the PDF document
documents = loader.load()

# Check the content of the loaded documents
for doc in documents:
    print(doc.page_content)

See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/257988758
Futu re Studies and Research in Egypt – Overview, Examples, Perspectives
Book  · Februar y 2012
CITATIONS
9READS
2,229
1 author:
Edg ar Göll
Instit ute for F utures St udies and T echnolog y Assessment
203 PUBLICA TIONS    338 CITATIONS    
SEE PROFILE
All c ontent f ollo wing this p age was uplo aded b y Edg ar Göll  on 04 June 2014.
The user has r equest ed enhanc ement of the do wnlo aded file.
IZT 
 Institut für Zukunftsstudien und Technologiebewertung  
 Institute for Futures Studies and Technology Assessment  
 
 
 
 
 
 
 
 
 
 
 
 
 
  
 
 
 
 
 
 
 
Berlin, February 2012  
ISBN 978 -3-941374 -17-1 
 2012 IZT 
 Future Studies and Research  
in Egypt –  
Overview, Examples, Perspectives  
 
 
Dr. Edgar Göll  
 
WerkstattBericht  Nr. 117  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
2 Futur e Studies and Research in Egypt  
Bibliografische Information Der Deutschen Bi

### Split documents:

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                            chunk_overlap=200,
                                            length_function=len,
                                            separators=["\n\n", "\n", " "])
chunks = text_splitter.split_documents(documents)

chunks

[Document(metadata={'source': 'C:\\Users\\ahmed\\Downloads\\Documents\\IZT_WB117.pdf', 'page': 0}, page_content='See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/257988758\nFutu re Studies and Research in Egypt – Overview, Examples, Perspectives\nBook  · Februar y 2012\nCITATIONS\n9READS\n2,229\n1 author:\nEdg ar Göll\nInstit ute for F utures St udies and T echnolog y Assessment\n203 PUBLICA TIONS \xa0\xa0\xa0338 CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll c ontent f ollo wing this p age was uplo aded b y Edg ar Göll  on 04 June 2014.\nThe user has r equest ed enhanc ement of the do wnlo aded file.'),
 Document(metadata={'source': 'C:\\Users\\ahmed\\Downloads\\Documents\\IZT_WB117.pdf', 'page': 1}, page_content='IZT \n Institut für Zukunftsstudien und Technologiebewertung  \n Institute for Futures Studies and Technology Assessment  \n \n \n \n \n \n \n \n \n \n \n \n \n \n  \n \n \n \n \n \n \n \nBerlin, February 2012  \

In [3]:
len(chunks)

118

In [4]:
print(chunks[24])

page_content='presented. The major subchapter will describe a selection of institutions, organis ations and experts in 
the fi eld of future research in Egypt . These are the Center for Future Studies  at the IDSC (CFS), the 
(ACPSS), the IFS in Assiut, the N ational Research Center (NRC) and at the end of the chapter a brief 
overview of others. As will be shown there are very few “future research institutions”. Most of the 
mentioned institutions have only a small team or even single experts who are engaged in future 
research or serious future thinking.  
3.1 Frustrated Beginnings in Egypt  
Looking into the future  has a long history, one prominent example from ancient Egypt has been 
mentioned in chapter 2 above. Tha t example shows an important feature of future thinking: it is not a 
trivial activitiy and it has the function to inform and to help to legitimate decisions. Therefore it is 
often a source for power, at least an instrument for decision -makers. Modern futur e thinki

### get embeddings:

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from Helper.config import get_config

embedding_model=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=get_config().gemini_key)

c:\Users\ahmed\Downloads\Courses\DEBI\My Projects\Grad-Project-RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
sentences = ["This is an example sentence", "Each sentence is converted"]
embeddings =embedding_model.embed_documents(sentences)
print(embeddings)

[[0.04291701316833496, -0.017652301117777824, -0.05807189270853996, -0.006633734796196222, 0.06482633948326111, 0.027622712776064873, 0.02355710044503212, -0.03398915380239487, -0.01315692625939846, 0.034130118787288666, -0.016170140355825424, 0.02988377958536148, 0.0037810313515365124, 0.022713758051395416, -0.021359501406550407, -0.026215320453047752, 0.02058870904147625, 0.017199700698256493, 0.0331479012966156, -0.014784585684537888, 0.0010762796737253666, 0.007957623340189457, -0.0157819502055645, 0.006626583635807037, 0.031216345727443695, -0.02195117063820362, 0.018550431355834007, -0.03645562008023262, -0.02541123703122139, 0.009106612764298916, -0.023128660395741463, 0.03818253055214882, -0.052823394536972046, 0.015865812078118324, 0.0035584981087595224, -0.04072296619415283, -0.004932435695081949, 0.019135834649205208, -0.005131716374307871, 0.0032915801275521517, 0.005523574072867632, -0.028399838134646416, -0.03707527741789818, 0.00275097182020545, 0.011153154075145721, -0.

### build  vector database

In [7]:
import uuid
from langchain.vectorstores import Chroma

def create_vectorstore(chunks, embedding_function, vectorstore_path='data/'):

    # Create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
    # Ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []
    
    unique_chunks = [] 
    for chunk, id in zip(chunks, ids):     
        if id not in unique_ids:       
            unique_ids.add(id)
            unique_chunks.append(chunk) 

    # Create a new Chroma database from the documents
    vectorstore = Chroma.from_documents(documents=unique_chunks, 
                                        ids=list(unique_ids),
                                        embedding=embedding_function, 
                                        persist_directory = vectorstore_path)

    vectorstore.persist()

In [8]:
vectorstore = create_vectorstore(chunks,embedding_model)

C:\Users\ahmed\AppData\Local\Temp\ipykernel_10372\2910621263.py:25: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


pdf2vdb:

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from Helper.config import get_config

embedding_model=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=get_config().gemini_key)


# Path to your PDF file
pdf_path = r"C:\Users\ahmed\Downloads\Documents\IZT_WB117.pdf"

# Initialize the PDF Loader
loader = PyPDFLoader(pdf_path)

# Load the PDF document
documents = loader.load()
def processing(pdf_path,embedding_model):

    #loading_pdf
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    
    #Chunking pdfs
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                            chunk_overlap=200,
                                            length_function=len,
                                            separators=["\n\n", "\n", " "])
    chunks = text_splitter.split_documents(documents)

### trying lama 3.2

In [11]:
!pip install groq


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for groq from https://files.pythonhosted.org/packages/94/c3/8bf3cd2987e262f9fd45024313dae1d009edaa7a5b429566edfdaedaa024/groq-0.11.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/106.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/106.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/106.5 kB ? eta -:--:--
   --- ------------------------------------ 10.2/106.5 kB ? eta -:--:--
   -------------- ------------------------ 41.0/106.5 kB 960.0 kB/s eta 0:00:01
   ---------------------------------- ----- 92.2/106.5 kB 1.1 MB/s eta 0:00:01
   ---------------------------------- ----- 92.2/106.5 kB 1.1 MB/s eta 0:00:01
   ------------------------------------ - 102.4/106.5 kB 454.0 kB/s eta 0:00:01
   -------------------------------------- 106.5/106.5 kB 439.6 kB/s eta 0:00:00


In [14]:
from langchain_groq import ChatGroq
from Helper import get_config
llm = ChatGroq(model="llama-3.1-70b-versatile",
               api_key=,
               temperature=0.6,
               max_tokens=1024)

The capital of Egypt is Cairo (Arabic: القاهرة, al-Qāhirah).

### testing funcations

In [5]:
from Scripts import *


documents=get_pdf_text('pdfs\s00125-004-1625-y.pdf')

In [6]:
documents

[Document(metadata={'source': 'pdfs\\s00125-004-1625-y.pdf', 'page': 0}, page_content='Diabetologia (2005) 48: 210 –214\nDOI 10.1007/s00125-004-1625-y\nSHORT COMMUNICATION\nT. Drivsholm .N. de Fine Olivarius .A. B. S. Nielsen .\nV. Siersma\nSymptoms, signs and complications in newly diagnosed type\n2 diabetic patients, and their relationship to glycaemia,blood pressure and weight\nReceived: 19 April 2004 / Accepted: 22 October 2004 / Published online: 14 January 2005\n#Springer-Verlag 2005\nAbstract Aims/hypothesis: To document the prevalence\nof typical diabetic symptoms, signs and complications inthe diagnosis of type 2 diabetes mellitus, examine their pre-diagnostic duration, and analyse associations with glycae-mic level, blood pressure (BP), and weight. Methods: An\nepidemiological population-based study of 1137 Danishpatients with type 2 diabetes newly diagnosed by generalpractitioners (GPs). GPs and patients together filled in aquestionnaire about typical symptoms, signs and com

In [2]:
vectorstore=create_vectorstore_from_texts(documents,'data')

c:\Users\ahmed\Downloads\Courses\DEBI\My Projects\Grad-Project-RAG\Scripts\script.py:134: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [3]:
vectorstore=load_vectorstore('data')

c:\Users\ahmed\Downloads\Courses\DEBI\My Projects\Grad-Project-RAG\Scripts\script.py:174: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  return Chroma(persist_directory=vectorstore_path,


In [4]:
query_document(vectorstore,query="How does the correlation between intelligence and large vocabularies, as described by Spearman (1904), relate to the perception of an author's intelligence?")

'According to the context, the correlation between intelligence and large vocabularies, as described by Spearman (1904), suggests that displaying a large vocabulary may provide cues that an author is intelligent. This is because intelligence and large vocabularies are positively correlated, which may lead readers to believe that an author is smart if they use complex vocabulary.'